In [15]:
import wandb
import pandas as pd
from PIL import Image, ImageOps
import json
import numpy as np


with wandb.init(project='diameterY', mode='online') as run:
    artifact = wandb.Artifact('rendered-fibers-mini-preview', type='table-dataset', description='rendered-fibers in table version')
    # run.use_artifact('rendered-fibers-table:latest')
    # artifact.download()
    df = pd.DataFrame(columns=['image', 'diameters', 'p1(s)', 'p2(s)', 'set'])
    for set_name, size in {'train':20,'val':20,'test':20}.items():
        for i in range(size):
            basename = f'../output/{set_name}{i:04d}'
            image = Image.open(f'{basename}.png')
            image = ImageOps.grayscale(image)
            image = np.array(image)
            # print(image.shape)
            seg = np.squeeze(np.load(f'{basename}_seg.npz')['y'])
            seg = np.max([np.zeros_like(seg, dtype=np.int32), seg.astype(np.int32)-99], axis=0).astype(int)
            with open(f'{basename}.json') as file:
                params = json.load(file)
            
            class_labels = {int(k)-99:f'f{int(k)-99:02d}' for k in params.keys() if k!='0'}
            class_labels.update({0:'bg'})
            wandb_im = wandb.Image(image, caption=basename, masks={
                'ground_truth':{
                    'mask_data': seg,
                    'class_labels': class_labels  
                }
            })
            vals = list(params.values())[1:]
            df = df.append({
                'image':wandb_im,
                'diameters': [v['d'] for v in vals],
                'p1(s)': [v['p1'] for v in vals],
                'p2(s)': [v['p2'] for v in vals],
                'set':set_name
            }, ignore_index=True)
    
    run.log({'preview_table':wandb.Table(dataframe=df)})
            




/tmp/ipykernel_905982/1697777281.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/tmp/ipykernel_905982/1697777281.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/tmp/ipykernel_905982/1697777281.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/tmp/ipykernel_905982/1697777281.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/tmp/ipykernel_905982/1697777281.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/tmp/ipykernel_905982/1697777281.py:34: 